In [ ]:
import time
from pathlib import Path

from crow_client import CrowClient
from crow_client.models import (
    AuthType,
    CrowDeploymentConfig,
    DockerContainerConfiguration,
    FramePath,
    JobRequest,
    RuntimeConfig,
    Stage,
)
from ldp.agent import AgentConfig

In [ ]:
# Instantiate the client
client = CrowClient(stage=Stage.DEV, auth_type=AuthType.GOOGLE)

127.0.0.1 - - [14/Mar/2025 10:10:41] "GET /?state=fQHtLyC5T4rSkWU2wDyMHg&code=4%2F0AQSTgQE_M3CZS_5e0Gvv5Vkpp7r21iSX14wmO_LBi-ZtBF0rSRFfj9a7yzI2QJPAuOiJQA&scope=openid&authuser=0&prompt=consent HTTP/1.1" 200 -


In [ ]:
# Create a simple Job
job_data = JobRequest(
    name="job-futurehouse-dummy-env-dev",
    query="How many moons does earth have?",
)
client.create_job(job_data)

while client.get_job()["status"] != "success":
    time.sleep(5)
print(client.get_job())

{'status': 'success', 'public': False, 'agent_state': [{'id': 'dc6b256c-d11f-4c60-8715-2a322362a524', 'trajectory_id': '5f36570b-bebb-44ce-8984-245097f0a286', 'trajectory_timestep': 4, 'step': 'after_transition', 'state': {'transition': {'timestep': 4, 'agent_state': {'tools': [{'type': 'function', 'info': {'name': 'print_story', 'description': 'Print a story.', 'parameters': {'type': 'object', 'properties': {'story': {'description': 'Story to print.', 'title': 'Story', 'type': 'string'}}, 'required': ['story']}}}, {'type': 'function', 'info': {'name': 'cast_float', 'description': 'Cast the input argument x to a float.', 'parameters': {'type': 'object', 'properties': {'x': {'title': 'X', 'type': 'string'}}, 'required': ['x']}}}, {'type': 'function', 'info': {'name': 'cast_int', 'description': 'Cast the input argument x to an integer.', 'parameters': {'type': 'object', 'properties': {'x': {'title': 'X', 'type': 'number'}}, 'required': ['x']}}}], 'messages': [{'role': 'user', 'content': 

In [ ]:
# Create a more complex job with runtime config

agent = AgentConfig(
    agent_type="ReActAgent",
    agent_kwargs={
        "model": "gpt-4o-mini",
        "temperature": 0.0,
    },
)
job_data = JobRequest(
    name="job-futurehouse-dummy-env-dev",
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=5),
)
client.create_job(job_data)

while client.get_job()["status"] != "success":
    time.sleep(5)
print(client.get_job())

{'status': 'success', 'public': False, 'agent_state': [{'id': 'fcd05d1d-660a-41a0-85de-6a42259d5db7', 'trajectory_id': '75b5c39a-ff9d-40a7-aca9-84af93e99079', 'trajectory_timestep': 4, 'step': 'after_transition', 'state': {'transition': {'timestep': 4, 'agent_state': {'tools': [{'type': 'function', 'info': {'name': 'print_story', 'description': 'Print a story.', 'parameters': {'type': 'object', 'properties': {'story': {'description': 'Story to print.', 'title': 'Story', 'type': 'string'}}, 'required': ['story']}}}, {'type': 'function', 'info': {'name': 'cast_float', 'description': 'Cast the input argument x to a float.', 'parameters': {'type': 'object', 'properties': {'x': {'title': 'X', 'type': 'string'}}, 'required': ['x']}}}, {'type': 'function', 'info': {'name': 'cast_int', 'description': 'Cast the input argument x to an integer.', 'parameters': {'type': 'object', 'properties': {'x': {'title': 'X', 'type': 'number'}}, 'required': ['x']}}}], 'messages': [{'role': 'user', 'content': 

In [ ]:
# Deploy a Crow with Frame Paths
frame_paths = [
    FramePath(path="state.pdbs", type="pdb", is_iterable=True),
    FramePath(path="state.single_pdb", type="pdb"),
]

crow = CrowDeploymentConfig(
    path=Path("./envs/dummy_env"),
    environment="dummy_env.env.DummyEnv",
    requires_aviary_internal=False,
    environment_variables={"SAMPLE_ENV_VAR": "sample_val"},
    agent="ldp.agent.SimpleAgent",
    container_config=DockerContainerConfiguration(cpu="1", memory="2Gi"),
    force=True,
    frame_paths=frame_paths,
    task_description="This is a dummy task",
)
client.create_crow(crow)


Available organizations:
[1]. FutureHouse
[2]. MaykPersonal


Overwriting existing deployment '@MaykPersonal/dummy-env-dev'


{'build_id': '2bf5b4ec-9804-45c8-82ab-ecc1c1dbef20',
 'image_uri': 'us-west2-docker.pkg.dev/agents-441015/crow-job-dev/job-maykpersonal-dummy-env-dev:timestamp-20250314-171547',
 'logs_uri': 'https://console.cloud.google.com/cloud-build/builds;region=us-west2/2bf5b4ec-9804-45c8-82ab-ecc1c1dbef20?project=209212876053',
 'job_name': 'job-maykpersonal-dummy-env-dev'}

In [ ]:
while client.get_build_status()["status"] != "SUCCESS":
    time.sleep(5)
print(client.get_build_status())

HTTPStatusError: Client error '403 Forbidden' for url 'https://dev.api.platform.futurehouse.org/v0.1/builds/2bf5b4ec-9804-45c8-82ab-ecc1c1dbef20'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403